<a href="https://colab.research.google.com/github/DanishFaraaz/Deep-Learning-Projects/blob/main/8_Pytorch_Fashion_MNIST_With_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import PIL

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
import torch.nn as nn

print("GPU Available: {}".format(torch.cuda.is_available()))
device = 'cuda'

GPU Available: True


**Using Transforms for Data Augmentation**

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomAffine(degrees=10, translate=(0.05,0.05), shear=5),
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15, interpolation=PIL.Image.BILINEAR),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ),(0.5, ))
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, ),(0.5, ))
    ])
}

In [ ]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=data_transforms['train'])
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=data_transforms['val'])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

100%|██████████| 26421880/26421880 [00:02<00:00, 10530919.76it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 174107.39it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3238062.27it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7533941.73it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3)
    self.conv1_bn = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.conv2_bn = nn.BatchNorm2d(64)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(64 * 12 * 12, 128)
    self.fc2 = nn.Linear(128, 10)
    self.dropOut = nn.Dropout(0.2)

  def forward(self, x):
    x = F.relu(self.conv1_bn(self.conv1(x)))
    x = self.dropOut(x)
    x = self.dropOut(F.relu(self.conv2_bn(self.conv2(x))))
    x = self.pool(x)
    x = x.view(-1, 64 * 12 * 12)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropOut): Dropout(p=0.2, inplace=False)
)

**Adding L2 Regularization**

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

In [ ]:
epochs = 15

epoch_log = []
loss_log = []
accuracy_log = []

for epoch in range(epochs):
  print(f'Staring Epoch: {epoch+1}....')

  running_loss = 0.0

  for i, data in enumerate(trainloader, 0):
    inputs, labels = data

    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i%100==99:
      correct = 0
      total = 0

      with torch.no_grad():
        for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted==labels).sum().item()

        accuracy = 100 * correct / total
        epoch_num = epoch + 1
        actual_loss = running_loss / 50
        print(f'Epoch : {epoch_num}, Mini-Batches Completed: {(i + 1)}, Loss: {actual_loss:.3f}, Test Accuracy = {accuracy:.3f}%')
        running_loss = 0.0

  epoch_log.append(epoch_num)
  loss_log.append(actual_loss)
  accuracy_log.append(accuracy)

print("Finished Training")

Staring Epoch: 1....
Epoch : 1, Mini-Batches Completed: 100, Loss: 2.510, Test Accuracy = 70.910%
Epoch : 1, Mini-Batches Completed: 200, Loss: 1.553, Test Accuracy = 75.810%
Epoch : 1, Mini-Batches Completed: 300, Loss: 1.407, Test Accuracy = 75.790%
Epoch : 1, Mini-Batches Completed: 400, Loss: 1.419, Test Accuracy = 76.940%
Epoch : 1, Mini-Batches Completed: 500, Loss: 1.314, Test Accuracy = 77.500%
Epoch : 1, Mini-Batches Completed: 600, Loss: 1.203, Test Accuracy = 79.560%
Epoch : 1, Mini-Batches Completed: 700, Loss: 1.236, Test Accuracy = 80.470%
Epoch : 1, Mini-Batches Completed: 800, Loss: 1.246, Test Accuracy = 78.880%
Epoch : 1, Mini-Batches Completed: 900, Loss: 1.204, Test Accuracy = 80.950%
Epoch : 1, Mini-Batches Completed: 1000, Loss: 1.226, Test Accuracy = 81.650%
Epoch : 1, Mini-Batches Completed: 1100, Loss: 1.106, Test Accuracy = 81.850%
Epoch : 1, Mini-Batches Completed: 1200, Loss: 1.084, Test Accuracy = 81.300%
Epoch : 1, Mini-Batches Completed: 1300, Loss: 1.067

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for data in testloader:
    images, labels = data

    images = images.to(device)
    labels = labels.to(device)

    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on 10,000 images is: {accuracy:.4}%')

NameError: ignored

In [ ]:
fig, ax1 = plt.subplots()

plt.title('Accuracy & Loss vs Epoch Mini-Batches')
plt.xticks(rotation=45)

ax2 = ax1.twinx()

ax1.plot(epoch_log, loss_log, 'g-')
ax2.plot(epoch_log, accuracy_log, 'b-')

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='g')
ax2.set_ylabel('Test Accuracy', color='b')

plt.show()